# FNNs com Keras

A biblioteca Keras fornece uma API padrão de alto nível para acessar o tensorflow. A seguir vamos ver como implementar algumas das técnicas anteriores usando Keras. Note que a API implementada na aula anterior foi inspirada na do Keras, modo sequencial, que é descrita a seguir, usando exemplos.

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, \
    Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer

/home/marco/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Como antes, vamos usar a coleção MNIST. Desta vez, vamos ter tantos neurônios na camada de saída quanto as classes em MNIST (10). Assim, vamos usar codificação _one-hot_ na classe.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/MNIST_data', one_hot=True)

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


Inicialmente, vamos criar uma arquitetura larga. Ela tem 250 neurônios em uma única camada escondida. Ela será treinada em 5 épocas com mini-batches de tamanho 128.

In [3]:
# define vars
input_num_units = 28*28
hidden_num_units = 250
output_num_units = 10
epochs = 5
batch_size = 128

model = Sequential()
model.add(Dense(units=hidden_num_units, 
       input_dim=input_num_units, activation='relu'))
model.add(Dense(units=output_num_units, 
       input_dim=hidden_num_units, activation='softmax'))

Vamos treiná-la usando como função de perda a entropia cruzada, com algoritmo de otimização Adam (gradiente descendente com momento) e avaliá-la em termos de acurácia:

In [5]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

trained_model_500 = model.fit(mnist.train.images, mnist.train.labels, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              validation_data=(mnist.validation.images, 
                                               mnist.validation.labels))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 1s - loss: 0.0464 - acc: 0.9860 - val_loss: 0.0721 - val_acc: 0.9770
Epoch 2/5
55000/55000 [==============================] - 1s - loss: 0.0338 - acc: 0.9902 - val_loss: 0.0686 - val_acc: 0.9774
Epoch 3/5
55000/55000 [==============================] - 1s - loss: 0.0265 - acc: 0.9928 - val_loss: 0.0650 - val_acc: 0.9816
Epoch 4/5
55000/55000 [==============================] - 1s - loss: 0.0200 - acc: 0.9948 - val_loss: 0.0635 - val_acc: 0.9780
Epoch 5/5
55000/55000 [==============================] - 1s - loss: 0.0160 - acc: 0.9961 - val_loss: 0.0664 - val_acc: 0.9796


Uma dúvida que surge é se uma rede profunda, com o mesmo número de nerônios ocultos, porém através de 5 camadas, é capaz de algo melhor. É o que vamos ver na arquitetura seguinte.

In [6]:
# define vars
input_num_units = 784
hidden1_num_units = 50
hidden2_num_units = 50
hidden3_num_units = 50
hidden4_num_units = 50
hidden5_num_units = 50
output_num_units = 10

epochs = 5
batch_size = 128

model = Sequential([
 Dense(units=hidden1_num_units, 
       input_dim=input_num_units, activation='relu'),
 Dense(units=hidden2_num_units, 
       input_dim=hidden1_num_units, activation='relu'),
 Dense(units=hidden3_num_units, 
       input_dim=hidden2_num_units, activation='relu'),
 Dense(units=hidden4_num_units, 
       input_dim=hidden3_num_units, activation='relu'),
 Dense(units=hidden5_num_units, 
       input_dim=hidden4_num_units, activation='relu'),
 Dense(units=output_num_units, 
       input_dim=hidden5_num_units, activation='softmax'),
 ])

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [7]:
trained_model_5d = model.fit(mnist.train.images, mnist.train.labels, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              validation_data=(mnist.validation.images, 
                                               mnist.validation.labels))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 2s - loss: 0.4306 - acc: 0.8651 - val_loss: 0.1721 - val_acc: 0.9498
Epoch 2/5
55000/55000 [==============================] - 2s - loss: 0.1613 - acc: 0.9513 - val_loss: 0.1354 - val_acc: 0.9612
Epoch 3/5
55000/55000 [==============================] - 2s - loss: 0.1204 - acc: 0.9633 - val_loss: 0.1265 - val_acc: 0.9636
Epoch 4/5
55000/55000 [==============================] - 2s - loss: 0.0956 - acc: 0.9709 - val_loss: 0.0989 - val_acc: 0.9704
Epoch 5/5
55000/55000 [==============================] - 2s - loss: 0.0801 - acc: 0.9749 - val_loss: 0.0974 - val_acc: 0.9714


Uma forma de melhorar o resultado talvez seja usar Dropout. Vamos ver como se comporta, usando uma taxa de droput de 20%:

In [8]:
# define vars
input_num_units = 784
hidden1_num_units = 50
hidden2_num_units = 50
hidden3_num_units = 50
hidden4_num_units = 50
hidden5_num_units = 50
output_num_units = 10

epochs = 5
batch_size = 128

dropout_ratio = 0.2

model = Sequential([
 Dense(units=hidden1_num_units, 
       input_dim=input_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden2_num_units, 
       input_dim=hidden1_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden3_num_units, 
       input_dim=hidden2_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden4_num_units, 
       input_dim=hidden3_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden5_num_units, 
       input_dim=hidden4_num_units, activation='relu'),
 Dropout(dropout_ratio),

Dense(units=output_num_units, 
      input_dim=hidden5_num_units, activation='softmax'),
 ])

In [9]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

trained_model_5d_drop = model.fit(mnist.train.images, mnist.train.labels, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              validation_data=(mnist.validation.images, 
                                               mnist.validation.labels))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 3s - loss: 0.9269 - acc: 0.6910 - val_loss: 0.2732 - val_acc: 0.9236
Epoch 2/5
55000/55000 [==============================] - 2s - loss: 0.4047 - acc: 0.8894 - val_loss: 0.2096 - val_acc: 0.9452
Epoch 3/5
55000/55000 [==============================] - 2s - loss: 0.3231 - acc: 0.9154 - val_loss: 0.1707 - val_acc: 0.9530
Epoch 4/5
55000/55000 [==============================] - 2s - loss: 0.2811 - acc: 0.9268 - val_loss: 0.1548 - val_acc: 0.9598
Epoch 5/5
55000/55000 [==============================] - 2s - loss: 0.2561 - acc: 0.9331 - val_loss: 0.1465 - val_acc: 0.9632


O desempenho piorou. Uma razão possível talvez seja o fato de que não estamos treinando o suficiente. Assim, vamos aumentar o número de épocas.

In [10]:
# define vars
input_num_units = 784
hidden1_num_units = 50
hidden2_num_units = 50
hidden3_num_units = 50
hidden4_num_units = 50
hidden5_num_units = 50
output_num_units = 10

epochs = 50
batch_size = 128

dropout_ratio = 0.2

model = Sequential([
 Dense(units=hidden1_num_units, 
       input_dim=input_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden2_num_units, 
       input_dim=hidden1_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden3_num_units, 
       input_dim=hidden2_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden4_num_units, 
       input_dim=hidden3_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden5_num_units, 
       input_dim=hidden4_num_units, activation='relu'),
 Dropout(dropout_ratio),

Dense(units=output_num_units, 
      input_dim=hidden5_num_units, activation='softmax'),
 ])

In [11]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

trained_model_5d_drop_mais_epocas = model.fit(mnist.train.images, 
                                              mnist.train.labels, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              validation_data=(mnist.validation.images, 
                                               mnist.validation.labels))

Train on 55000 samples, validate on 5000 samples
Epoch 1/50
55000/55000 [==============================] - 3s - loss: 0.9274 - acc: 0.6877 - val_loss: 0.2709 - val_acc: 0.9262
Epoch 2/50
55000/55000 [==============================] - 2s - loss: 0.3926 - acc: 0.8932 - val_loss: 0.2016 - val_acc: 0.9404
Epoch 3/50
55000/55000 [==============================] - 2s - loss: 0.3175 - acc: 0.9164 - val_loss: 0.1765 - val_acc: 0.9520
Epoch 4/50
55000/55000 [==============================] - 2s - loss: 0.2756 - acc: 0.9272 - val_loss: 0.1560 - val_acc: 0.9574
Epoch 5/50
55000/55000 [==============================] - 2s - loss: 0.2502 - acc: 0.9345 - val_loss: 0.1518 - val_acc: 0.9576
Epoch 6/50
55000/55000 [==============================] - 2s - loss: 0.2381 - acc: 0.9371 - val_loss: 0.1516 - val_acc: 0.9614
Epoch 7/50
55000/55000 [==============================] - 1s - loss: 0.2193 - acc: 0.9421 - val_loss: 0.1491 - val_acc: 0.9626
Epoch 8/50
55000/55000 [==============================] - 2s -

E nada muito extraordinário. O próximo passo agora é testar uma rede larga e profunda (e lenta :(). Com ela, temos a chance de realmente nos aproximarmos do desempenho dos melhores métodos, baseados em convoluções. Para deixar as coisas menos complexas, vou deixar 500 neurônios na 1a camada e apenas 250 nas demais. Vou testar com 25 épocas... ah, pode esperar!

In [12]:
# define vars
input_num_units = 784
hidden1_num_units = 500
hidden2_num_units = 250
hidden3_num_units = 250
hidden4_num_units = 250
hidden5_num_units = 250
output_num_units = 10

epochs = 25
batch_size = 128

dropout_ratio = 0.2

model = Sequential([
 Dense(units=hidden1_num_units, 
       input_dim=input_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden2_num_units, 
       input_dim=hidden1_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden3_num_units, 
       input_dim=hidden2_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden4_num_units, 
       input_dim=hidden3_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(units=hidden5_num_units, 
       input_dim=hidden4_num_units, activation='relu'),
 Dropout(dropout_ratio),

Dense(units=output_num_units, 
      input_dim=hidden5_num_units, activation='softmax'),
 ])

In [13]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

trained_model_5d_larga_profunda = model.fit(mnist.train.images, 
                                              mnist.train.labels, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              validation_data=(mnist.validation.images, 
                                               mnist.validation.labels))

Train on 55000 samples, validate on 5000 samples
Epoch 1/25
55000/55000 [==============================] - 3s - loss: 0.3620 - acc: 0.8880 - val_loss: 0.1195 - val_acc: 0.9660
Epoch 2/25
55000/55000 [==============================] - 2s - loss: 0.1434 - acc: 0.9588 - val_loss: 0.0996 - val_acc: 0.9702
Epoch 3/25
55000/55000 [==============================] - 2s - loss: 0.1100 - acc: 0.9683 - val_loss: 0.0865 - val_acc: 0.9748
Epoch 4/25
55000/55000 [==============================] - 2s - loss: 0.0867 - acc: 0.9752 - val_loss: 0.0840 - val_acc: 0.9782
Epoch 5/25
55000/55000 [==============================] - 2s - loss: 0.0765 - acc: 0.9776 - val_loss: 0.0767 - val_acc: 0.9802
Epoch 6/25
55000/55000 [==============================] - 2s - loss: 0.0620 - acc: 0.9819 - val_loss: 0.0737 - val_acc: 0.9800
Epoch 7/25
55000/55000 [==============================] - 2s - loss: 0.0600 - acc: 0.9823 - val_loss: 0.0666 - val_acc: 0.9826
Epoch 8/25
55000/55000 [==============================] - 2s -

Este modelo já conseguiu um resultado bem superior ao anterior. Contudo, fica claro agora que já há algum overfitting, devido à complexidade do modelo. Vamos ver qual o efeito agora de aplicarmos _batch normalization_.

In [14]:
# define vars
input_num_units = 784
hidden1_num_units = 500
hidden2_num_units = 250
hidden3_num_units = 250
hidden4_num_units = 250
hidden5_num_units = 250
output_num_units = 10

epochs = 25
batch_size = 128

dropout_ratio = 0.2

from keras.layers.normalization import BatchNormalization

model = Sequential([
 Dense(units=hidden1_num_units, input_dim=input_num_units),
 BatchNormalization(), Activation('relu'),
 Dropout(dropout_ratio),
    
 Dense(units=hidden2_num_units, input_dim=hidden1_num_units),
 BatchNormalization(), Activation('relu'),
 Dropout(dropout_ratio),
    
 Dense(units=hidden3_num_units, input_dim=hidden2_num_units),
 BatchNormalization(), Activation('relu'),
 Dropout(dropout_ratio),
    
 Dense(units=hidden4_num_units, input_dim=hidden3_num_units),
 BatchNormalization(), Activation('relu'),
 Dropout(dropout_ratio),
    
 Dense(units=hidden5_num_units, input_dim=hidden4_num_units),
 BatchNormalization(), Activation('relu'),
 Dropout(dropout_ratio),
    
 Dense(units=output_num_units, input_dim=hidden5_num_units),
 BatchNormalization(), Activation('softmax'),
])

In [15]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

trained_model_5d_larga_profunda = model.fit(mnist.train.images, 
                                              mnist.train.labels, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              validation_data=(mnist.validation.images, 
                                               mnist.validation.labels))

Train on 55000 samples, validate on 5000 samples
Epoch 1/25
55000/55000 [==============================] - 7s - loss: 0.5520 - acc: 0.8895 - val_loss: 0.2106 - val_acc: 0.9598
Epoch 2/25
55000/55000 [==============================] - 5s - loss: 0.2484 - acc: 0.9532 - val_loss: 0.1225 - val_acc: 0.9734
Epoch 3/25
55000/55000 [==============================] - 5s - loss: 0.1769 - acc: 0.9640 - val_loss: 0.0976 - val_acc: 0.9768
Epoch 4/25
55000/55000 [==============================] - 5s - loss: 0.1373 - acc: 0.9696 - val_loss: 0.0820 - val_acc: 0.9794
Epoch 5/25
55000/55000 [==============================] - 4s - loss: 0.1124 - acc: 0.9740 - val_loss: 0.0765 - val_acc: 0.9814
Epoch 6/25
55000/55000 [==============================] - 5s - loss: 0.0980 - acc: 0.9766 - val_loss: 0.0691 - val_acc: 0.9808
Epoch 7/25
55000/55000 [==============================] - 5s - loss: 0.0855 - acc: 0.9789 - val_loss: 0.0675 - val_acc: 0.9818
Epoch 8/25
55000/55000 [==============================] - 5s -

Mas à frente, vamos melhorar mais este resultado usando uma rede convolutiva.

## Keras Funcional

A API sequencial do Keras permite a criação de modelos camada a camada, o que é uma boa representação para muitos modelos. Contudo, modelos mais complexos podem ter múltiplas entradas e saídas e várias sequências de camadas em paralelo. Isso não é simples de representar usando uma abstração sequencial. 

Uma abstração alternativa seria tratar a rede neural pelo o que ela é de fato: uma complexa combinação de funções. Esta é a ideia por trás da API funcional, que vamos ver agora.

### Uma FNN com Keras Funcional

Nosso primeiro modelo nesta aula foi:

```python
# define vars
input_num_units = 28*28
hidden_num_units = 250
output_num_units = 10
epochs = 5
batch_size = 128

model = Sequential()
model.add(Dense(units=hidden_num_units, 
       input_dim=input_num_units, activation='relu'))
model.add(Dense(units=output_num_units, 
       input_dim=hidden_num_units, activation='softmax'))
       
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

trained_model_500 = model.fit(mnist.train.images, mnist.train.labels, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              validation_data=(mnist.validation.images, 
                                               mnist.validation.labels))
```

Usando a notação funcional, ele seria re-escrito como:

In [26]:
from keras.layers import Input, Dense, Activation
from keras.models import Model

In [27]:
# define vars
input_num_units = 28*28
hidden_num_units = 250
output_num_units = 10
epochs = 5
batch_size = 128

A única parte do modelo que é diferente é realmente esta:

In [32]:
visible = Input(shape=(input_num_units,))
hidden = Dense(hidden_num_units, activation = 'relu')(visible)
output = Dense(output_num_units, activation = 'softmax')(hidden)
model = Model(inputs = visible, outputs = output)

# antes:
# model = Sequential()
# model.add(Dense(units=hidden_num_units, 
#       input_dim=input_num_units, activation='relu'))
# model.add(Dense(units=output_num_units, 
#       input_dim=hidden_num_units, activation='softmax'))

Ou seja, cada camada é tratada como uma função parametrizável. A saída de uma função é entrada para a próxima. A entrada é definida de forma explícita e o modelo é a composição das entradas e suas saídas (neste caso, apenas uma entrada e uma saída). Alternativamente, poderíamos definir este modelo assim:

In [29]:
visible = Input(shape=(input_num_units,))
hidden = Dense(hidden_num_units)(visible)
hidden = Activation('relu')(hidden)
output = Dense(output_num_units, activation = 'softmax')(hidden)
model = Model(inputs = visible, outputs = output)

Esta segunda formulação, em que a função de ativação é separada da camada oculta, é útil para executar operações antes da função de ativação como, por exemplo, BatchNormalization.

In [30]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [31]:
trained_model_500 = model.fit(mnist.train.images, mnist.train.labels, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              validation_data=(mnist.validation.images, 
                                               mnist.validation.labels))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 1s - loss: 0.3247 - acc: 0.9097 - val_loss: 0.1600 - val_acc: 0.9566
Epoch 2/5
55000/55000 [==============================] - 1s - loss: 0.1404 - acc: 0.9596 - val_loss: 0.1105 - val_acc: 0.9680
Epoch 3/5
55000/55000 [==============================] - 1s - loss: 0.0964 - acc: 0.9718 - val_loss: 0.0926 - val_acc: 0.9726
Epoch 4/5
55000/55000 [==============================] - 1s - loss: 0.0722 - acc: 0.9792 - val_loss: 0.0813 - val_acc: 0.9760
Epoch 5/5
55000/55000 [==============================] - 1s - loss: 0.0545 - acc: 0.9840 - val_loss: 0.0808 - val_acc: 0.9762


__Exercício__: Usando a API funcional do Keras, crie uma FNN com 10 neurônios de entrada, 3 camadas ocultas com 10, 20 e 10 neurônios, e uma camada de saída com 1 neurônio. Use _relu_ como função de ativação em cada camada oculta e _sigmoid_ para a camada de saída.

<div align="right">
<a href="#modelkf1" class="btn btn-default" data-toggle="collapse">Solução #1</a>
</div>
<div id="modelkf1" class="collapse">
```
from keras.models import Model
from keras.layers import Input, Dense

inp = Input(shape=(10,))
h1  = Dense(10, activation = 'relu')(inp)
h2  = Dense(20, activation = 'relu')(h1)
h3  = Dense(10, activation = 'relu')(h2)
out = Dense( 1, activation = 'sigmoid')(h3)

model = Model(inputs = inp, outputs = out)
```
</div>

### Combinando duas FNNs, um modelo de duas entradas

Imagine que queremos saber se um usuário de uma operadora de TV à cabo pretende ou não desistir da sua assinatura. Suponha que temos duas redes neurais que usam diferentes conjuntos de evidências para prever a desistência. A primeira tem como entrada a quantidade de tempo gasto pelo usuário em dez categorias de programas. A segunda tem como entrada 8 diferentes informações demográficas do usuário, como sexo, idade, faixa de renda, plano assinado, etc. Queremos usar as representações dos usuários aprendidas por estas duas redes para criar um classificador final que tem a seguinte arquitetura: 



<img src="images/FnnMerge.png" alt="FNN combinada" style="width: 300px;"/>

Abaixo temos uma representação da rede em Keras, que demonstra a flexibidade da API funcional:

In [33]:
# Multiple Inputs
from keras.models import Model
from keras.layers import Input, Dense 
from keras.layers.merge import concatenate

# first input model
in1  = Input(shape=(10,))
h11  = Dense(10, activation = 'relu')(in1)
h12  = Dense(15, activation = 'relu')(h11)
out1 = Dense(10, activation = 'relu')(h12)

# second input model
in2  = Input(shape=(8,))
h21  = Dense(8, activation = 'relu')(in2)
out2 = Dense(8, activation = 'relu')(h21)

# merge input models
merge = concatenate([out1, out2])

# interpretation model
hm1 = Dense(10, activation='relu')(merge)
hm2 = Dense(10, activation='relu')(hm1)
output = Dense(1, activation='sigmoid')(hm2)
model = Model(inputs=[in1, in2], outputs=output)

# summarize layers
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 10)            0                                            
____________________________________________________________________________________________________
input_7 (InputLayer)             (None, 10)            0                                            
____________________________________________________________________________________________________
dense_43 (Dense)                 (None, 10)            110         input_6[0][0]                    
____________________________________________________________________________________________________
dense_46 (Dense)                 (None, 10)            110         input_7[0][0]                    
___________________________________________________________________________________________

O treino e previsão para este modelo devem incruir dados para ambas as entradas, o que resultaria em algo assim:

```python
model.fit([treino_programacao, treino_demograficos], labels, epochs=10)
preds = model.predict([teste_programacao, teste_demograficos])
```

__Exercício__: Usando a API funcional do Keras, crie uma FNN que combina três outras FNNs. Cada uma das FNNs combinadas tem 10 neurônios de entrada e 2 camadas ocultas com 20 e 10 neurônios. As representações de saída de cada FNN são adicionadas (`keras.layers.merge.add`) umas às outras para formar a representação de entrada da rede de interpretação. Esta rede, por sua vez, é formada por três camadas ocultas de 10 neurônios e uma camada de saída com 5 neurônios. Use _relu_ como função de ativação em cada camada oculta e _sigmoid_ para a camada de saída. Quantos são os parâmetros treináveis desta rede?

<div align="right">
<a href="#modelkf2" class="btn btn-default" data-toggle="collapse">Solução #1</a>
</div>
<div id="modelkf2" class="collapse">
```
# Multiple Inputs
from keras.models import Model
from keras.layers import Input, Dense 
from keras.layers.merge import add

in1  = Input(shape=(20,))
h11  = Dense(20, activation = 'relu')(in1)
out1 = Dense(10, activation = 'relu')(h11)

in2  = Input(shape=(20,))
h21  = Dense(20, activation = 'relu')(in2)
out2 = Dense(10, activation = 'relu')(h21)

in3  = Input(shape=(20,))
h31  = Dense(20, activation = 'relu')(in3)
out3 = Dense(10, activation = 'relu')(h31)

# merge input models
merge = add([out1, out2, out3])

# interpretation model
hm1 = Dense(10, activation='relu')(merge)
hm2 = Dense(10, activation='relu')(hm1)
hm3 = Dense(10, activation='relu')(hm2)
output = Dense(5, activation='sigmoid')(hm3)
model = Model(inputs=[in1, in2, in3], outputs=output)

# summarize layers
print(model.summary())

# na minha conta, 2275 parâmetros.```
</div>

Nas próximas aulas, vamos ver mais exemplos de arquiteturas complexas.

Esta aula inclui material de <a href = "https://linkedin.com/in/luisotsm">Luis Otavio Silveira Martins</a>, <a href = "https://linkedin.com/in/erich-natsubori-sato"> Erich Natsubori Sato </a></h4>. Material adicional de Imanol Schlag (https://ischlag.github.io/2016/06/04/how-to-use-tensorboard/) e Faizan Shaikh (https://www.analyticsvidhya.com/blog/2016/10/tutorial-optimizing-neural-networks-using-keras-with-image-recognition-case-study/). A API funcional é baseada no material de Jason Brownlee, disponível em https://machinelearningmastery.com/keras-functional-api-deep-learning/.